# Modelo de predicción sismos EEUU

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely import wkt
from shapely.geometry import polygon
import numpy as np


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

# Lectura de archivos shapefile

Archivo principal (.shp): Este archivo contiene la geometría de los objetos geoespaciales, como puntos, líneas o polígonos, en formato binario.  

Archivo de índice (.shx): Este archivo es un índice de acceso espacial que permite un acceso más rápido a los datos en el archivo principal (.shp). Contiene información sobre la ubicación de cada registro en el archivo principal.  

Archivo de atributos (.dbf): Este archivo almacena los atributos o datos no espaciales asociados a los objetos geoespaciales. Puede contener información como nombres, valores, fechas u otra información relevante.  
 
Archivo de metadatos (.prj): Este archivo almacena la información del sistema de referencia de coordenadas (CRS) utilizado en el archivo Shapefile. El CRS define cómo se interpreta y proyecta la geometría espacial en un sistema de coordenadas específico.  

# MAPA CENTRO Y ESTE

In [2]:
#Leo el shapefile con geopandas
gdf_zonasCE = gpd.read_file('Shapefiles\CEUS_1PctIn1Yr_5Hz.shp')

In [3]:
gdf_zonasCE.head()

,ValueRange,geometry
0,40 - 60,"POLYGON ((-786984.399 3286941.888, -788200.330..."
1,30 - 40,"POLYGON ((-773036.783 3305960.178, -772758.019..."
2,20 - 30,"POLYGON ((-766738.972 3314078.833, -765464.024..."
3,16 - 20,"POLYGON ((-769309.282 3323964.882, -768067.743..."
4,12 - 16,"POLYGON ((-762096.479 3328986.210, -760806.454..."


In [4]:
gdf_zonasCE.ValueRange.unique()

array(['40 - 60', '30 - 40', '20 - 30', '16 - 20', '12 - 16', '8 - 12',
       '6 - 8', '4 - 6', '< 2', '> 80', '60 - 80', '2 - 4'], dtype=object)

In [5]:
gdf_zonasCE.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ValueRange  65 non-null     object  
 1   geometry    65 non-null     geometry
dtypes: geometry(1), object(1)
memory usage: 1.1+ KB


In [6]:
# Definir la proyección de destino (proyección geográfica en grados decimales)
crs_destino = 'EPSG:4326'  # WGS84, utilizado comúnmente en coordenadas geográficas

# Convertir el GeoDataFrame de zonas a la proyección de destino
gdf_zonasCE = gdf_zonasCE.to_crs(crs_destino)

# Verificar el nuevo formato de los polígonos
print(gdf_zonasCE['geometry'])

0     POLYGON ((-103.28787 31.11885, -103.30000 31.1...
1     POLYGON ((-103.15886 31.30000, -103.15537 31.2...
2     POLYGON ((-103.10000 31.37753, -103.08522 31.3...
3     POLYGON ((-103.13594 31.46406, -103.12207 31.4...
4     POLYGON ((-103.06452 31.51452, -103.05000 31.5...
                            ...                        
60    MULTIPOLYGON (((-111.50471 37.75000, -111.5151...
61    POLYGON ((-105.10897 35.45515, -105.10625 35.4...
62    MULTIPOLYGON (((-107.11831 36.35000, -107.1130...
63    POLYGON ((-65.00000 48.06236, -65.00000 24.600...
64    POLYGON ((-65.00000 48.06236, -65.00199 48.051...
Name: geometry, Length: 65, dtype: geometry


# MAPA OESTE 

In [7]:
gdf_zonasW = gpd.read_file('Shapefiles\WUS_1PctIn1Yr_5Hz.shp')

In [8]:
gdf_zonasW.head()

,ValueRange,geometry
0,20 - 30,"POLYGON ((-1662039.603 3115326.005, -1659795.4..."
1,30 - 40,"POLYGON ((-1821368.340 3277822.480, -1823606.9..."
2,20 - 30,"POLYGON ((-1727780.469 3394543.335, -1727247.2..."
3,12 - 16,"POLYGON ((-1861635.333 3477092.816, -1862862.4..."
4,30 - 40,"POLYGON ((-1937339.330 3488399.977, -1938269.8..."


In [9]:
gdf_zonasW.ValueRange.unique()

array(['20 - 30', '30 - 40', '12 - 16', '8 - 12', '6 - 8', '60 - 80',
       '4 - 6', '> 80', '< 2', '40 - 60', '16 - 20', '2 - 4'],
      dtype=object)

In [10]:
gdf_zonasW.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ValueRange  73 non-null     object  
 1   geometry    73 non-null     geometry
dtypes: geometry(1), object(1)
memory usage: 1.3+ KB


In [11]:
# Definir la proyección de destino (proyección geográfica en grados decimales)
crs_destino = 'EPSG:4326'  # WGS84, utilizado comúnmente en coordenadas geográficas

# Convertir el GeoDataFrame de zonas a la proyección de destino
gdf_zonasW = gdf_zonasW.to_crs(crs_destino)

# Verificar el nuevo formato de los polígonos
print(gdf_zonasW['geometry'])

0     POLYGON ((-112.03461 28.53461, -112.01204 28.5...
1     POLYGON ((-113.95503 29.70502, -113.97747 29.7...
2     POLYGON ((-113.23537 30.90000, -113.22420 30.8...
3     POLYGON ((-114.78820 31.38820, -114.80000 31.3...
4     POLYGON ((-115.59407 31.34407, -115.60000 31.3...
                            ...                        
68    MULTIPOLYGON (((-111.19531 39.04844, -111.1988...
69    MULTIPOLYGON (((-105.69219 38.15156, -105.6812...
70    POLYGON ((-116.78563 50.00000, -116.79933 49.9...
71    MULTIPOLYGON (((-124.78926 39.21074, -124.7870...
72    POLYGON ((-101.67384 29.44503, -101.67031 29.4...
Name: geometry, Length: 73, dtype: geometry


# Junto los dos geodataframes

In [12]:
# Unir dos GeoDataFrames
gdf_zonas= gdf_zonasCE.append(gdf_zonasW, ignore_index=True)

C:\Users\claud\AppData\Local\Temp\ipykernel_10536\3050802443.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf_zonas= gdf_zonasCE.append(gdf_zonasW, ignore_index=True)


In [13]:
gdf_zonas.head()

,ValueRange,geometry
0,40 - 60,"POLYGON ((-103.28787 31.11885, -103.30000 31.1..."
1,30 - 40,"POLYGON ((-103.15886 31.30000, -103.15537 31.2..."
2,20 - 30,"POLYGON ((-103.10000 31.37753, -103.08522 31.3..."
3,16 - 20,"POLYGON ((-103.13594 31.46406, -103.12207 31.4..."
4,12 - 16,"POLYGON ((-103.06452 31.51452, -103.05000 31.5..."


In [14]:
gdf_zonas.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ValueRange  138 non-null    object  
 1   geometry    138 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 2.3+ KB


# SISMOS

In [15]:
df = pd.read_csv('..\DASHBOARD\CSV_TRANSFORMADOS\Datos_USA.csv')

In [16]:
#Convierte las columnas de latitud y longitud en objetos de geometría de puntos utilizando la clase Point de la biblioteca shapely.
geometry = [Point(xy) for xy in zip(df['Longitud'], df['Latitud'])]
#Crea un GeoDataFrame a partir del DataFrame original y las geometrías de puntos.
gdf_sismos = gpd.GeoDataFrame(df, geometry=geometry)
#Utiliza el método to_file del GeoDataFrame para exportar el archivo Shapefile.
# output_shapefile = 'sismos_eeuu.shp'  # Nombre del archivo Shapefile de salida
# gdf_sismos.to_file(output_shapefile, driver='ESRI Shapefile')

C:\Users\claud\AppData\Local\Temp\ipykernel_10536\2662450449.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_sismos.to_file(output_shapefile, driver='ESRI Shapefile')


In [17]:
df.head()

,id,Magnitud,Place,Primer Registro,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,ids,nst,Dist Horizontal epicentro,RMS,Brecha azimutal,Longitud,Latitud,Profundidad,estado
0,nc73888821,2.83,58km WNW of Petrolia,2023-05-16 16:22:09.380,2023-05-17 02:32:10.393,NaN,NaN,NaN,0,123,",nc73888821,us6000kcbg,",21.0,0.58120,0.28,289.0,-124.955833,40.425335,-0.37,California
1,nc73888746,3.24,2km NNE of Almanor,2023-05-16 11:37:26.630,2023-05-16 23:58:52.060,18.0,4.3,NaN,0,169,",nc73888746,nn00859587,us6000kc9f,",42.0,0.02014,0.20,33.0,-121.167833,40.237167,6.02,California
2,ok2023jmlz,2.98,8 km NW of Prague,2023-05-15 21:30:20.910,2023-05-16 00:59:31.375,6.0,3.7,NaN,0,139,",ok2023jmlz,us6000kc5i,",90.0,0.00000,0.14,34.0,-96.749667,35.536000,7.81,Oklahoma
3,nc73888511,2.60,3km NNE of Prattville,2023-05-15 17:38:58.740,2023-05-15 23:58:18.995,2.0,2.0,NaN,0,104,",nc73888511,nn00859567,",36.0,0.01564,0.18,44.0,-121.142833,40.232167,6.76,California
4,nc73888396,2.63,41km W of Ferndale,2023-05-15 11:35:46.410,2023-05-15 17:11:12.082,NaN,NaN,NaN,0,106,",nc73888396,",25.0,0.47330,0.16,294.0,-124.746002,40.527000,20.25,California


In [18]:
gdf_sismos.head()

,id,Magnitud,Place,Primer Registro,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,ids,nst,Dist Horizontal epicentro,RMS,Brecha azimutal,Longitud,Latitud,Profundidad,estado,geometry
0,nc73888821,2.83,58km WNW of Petrolia,2023-05-16 16:22:09.380,2023-05-17 02:32:10.393,NaN,NaN,NaN,0,123,",nc73888821,us6000kcbg,",21.0,0.58120,0.28,289.0,-124.955833,40.425335,-0.37,California,POINT (-124.95583 40.42533)
1,nc73888746,3.24,2km NNE of Almanor,2023-05-16 11:37:26.630,2023-05-16 23:58:52.060,18.0,4.3,NaN,0,169,",nc73888746,nn00859587,us6000kc9f,",42.0,0.02014,0.20,33.0,-121.167833,40.237167,6.02,California,POINT (-121.16783 40.23717)
2,ok2023jmlz,2.98,8 km NW of Prague,2023-05-15 21:30:20.910,2023-05-16 00:59:31.375,6.0,3.7,NaN,0,139,",ok2023jmlz,us6000kc5i,",90.0,0.00000,0.14,34.0,-96.749667,35.536000,7.81,Oklahoma,POINT (-96.74967 35.53600)
3,nc73888511,2.60,3km NNE of Prattville,2023-05-15 17:38:58.740,2023-05-15 23:58:18.995,2.0,2.0,NaN,0,104,",nc73888511,nn00859567,",36.0,0.01564,0.18,44.0,-121.142833,40.232167,6.76,California,POINT (-121.14283 40.23217)
4,nc73888396,2.63,41km W of Ferndale,2023-05-15 11:35:46.410,2023-05-15 17:11:12.082,NaN,NaN,NaN,0,106,",nc73888396,",25.0,0.47330,0.16,294.0,-124.746002,40.527000,20.25,California,POINT (-124.74600 40.52700)


In [19]:
gdf_sismos.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 142593 entries, 0 to 142592
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   id                         142593 non-null  object  
 1   Magnitud                   142593 non-null  float64 
 2   Place                      142490 non-null  object  
 3   Primer Registro            142593 non-null  object  
 4   Ultimo registro            142593 non-null  object  
 5   Felt                       23869 non-null   float64 
 6   cdi                        23869 non-null   float64 
 7   mmi                        6038 non-null    float64 
 8   Posibilidad tsunami        142593 non-null  int64   
 9   Importancia del evento     142593 non-null  int64   
 10  ids                        142593 non-null  object  
 11  nst                        120414 non-null  float64 
 12  Dist Horizontal epicentro  93371 non-null   float64 
 13  RMS   

# Idea  
Con poligono.contains(punto), polygon y points genero una columna con el RangeValue del geodataframe de las zonas correspondientes

In [20]:
# Verifica el CRS de los GeoDataFrames
print(gdf_sismos.crs)
print(gdf_zonas.crs)

None
EPSG:4326


El resultado indica que el GeoDataFrame de los sismos (gdf_sismos) no tiene definido un sistema de referencia espacial (CRS), mientras que el GeoDataFrame de las zonas de riesgo (gdf_zonas) tiene un CRS definido.

In [21]:
# Asigna el CRS al GeoDataFrame de los sismos
gdf_sismos.set_crs(gdf_zonas.crs, inplace=True)

print(gdf_sismos.crs)
print(gdf_zonas.crs)


EPSG:4326
EPSG:4326


In [22]:
# Establece la columna de geometría en el GeoDataFrame de los sismos
gdf_sismos.set_geometry('geometry', inplace=True)

# Realiza la unión espacial
gdf_sismos_con_zonas = gpd.sjoin(gdf_sismos, gdf_zonas, how='left')

In [23]:
gdf_sismos_con_zonas.head()

,id,Magnitud,Place,Primer Registro,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,...,Dist Horizontal epicentro,RMS,Brecha azimutal,Longitud,Latitud,Profundidad,estado,geometry,index_right,ValueRange
0,nc73888821,2.83,58km WNW of Petrolia,2023-05-16 16:22:09.380,2023-05-17 02:32:10.393,NaN,NaN,NaN,0,123,...,0.58120,0.28,289.0,-124.955833,40.425335,-0.37,California,POINT (-124.95583 40.42533),109,40 - 60
1,nc73888746,3.24,2km NNE of Almanor,2023-05-16 11:37:26.630,2023-05-16 23:58:52.060,18.0,4.3,NaN,0,169,...,0.02014,0.20,33.0,-121.167833,40.237167,6.02,California,POINT (-121.16783 40.23717),106,20 - 30
2,ok2023jmlz,2.98,8 km NW of Prague,2023-05-15 21:30:20.910,2023-05-16 00:59:31.375,6.0,3.7,NaN,0,139,...,0.00000,0.14,34.0,-96.749667,35.536000,7.81,Oklahoma,POINT (-96.74967 35.53600),34,60 - 80
3,nc73888511,2.60,3km NNE of Prattville,2023-05-15 17:38:58.740,2023-05-15 23:58:18.995,2.0,2.0,NaN,0,104,...,0.01564,0.18,44.0,-121.142833,40.232167,6.76,California,POINT (-121.14283 40.23217),106,20 - 30
4,nc73888396,2.63,41km W of Ferndale,2023-05-15 11:35:46.410,2023-05-15 17:11:12.082,NaN,NaN,NaN,0,106,...,0.47330,0.16,294.0,-124.746002,40.527000,20.25,California,POINT (-124.74600 40.52700),109,40 - 60


In [24]:
# # Guardar el GeoDataFrame como CSV
# gdf_sismos_con_zonas.to_csv('Sismos_usa_ML.csv', index=False)

In [25]:
# # Guardar el GeoDataFrame como CSV
# gdf_sismos_con_zonas.to_file("Sismos_usa_ML.json", driver="GeoJSON")

In [26]:
gdf_sismos_con_zonas.head()

,id,Magnitud,Place,Primer Registro,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,...,Dist Horizontal epicentro,RMS,Brecha azimutal,Longitud,Latitud,Profundidad,estado,geometry,index_right,ValueRange
0,nc73888821,2.83,58km WNW of Petrolia,2023-05-16 16:22:09.380,2023-05-17 02:32:10.393,NaN,NaN,NaN,0,123,...,0.58120,0.28,289.0,-124.955833,40.425335,-0.37,California,POINT (-124.95583 40.42533),109,40 - 60
1,nc73888746,3.24,2km NNE of Almanor,2023-05-16 11:37:26.630,2023-05-16 23:58:52.060,18.0,4.3,NaN,0,169,...,0.02014,0.20,33.0,-121.167833,40.237167,6.02,California,POINT (-121.16783 40.23717),106,20 - 30
2,ok2023jmlz,2.98,8 km NW of Prague,2023-05-15 21:30:20.910,2023-05-16 00:59:31.375,6.0,3.7,NaN,0,139,...,0.00000,0.14,34.0,-96.749667,35.536000,7.81,Oklahoma,POINT (-96.74967 35.53600),34,60 - 80
3,nc73888511,2.60,3km NNE of Prattville,2023-05-15 17:38:58.740,2023-05-15 23:58:18.995,2.0,2.0,NaN,0,104,...,0.01564,0.18,44.0,-121.142833,40.232167,6.76,California,POINT (-121.14283 40.23217),106,20 - 30
4,nc73888396,2.63,41km W of Ferndale,2023-05-15 11:35:46.410,2023-05-15 17:11:12.082,NaN,NaN,NaN,0,106,...,0.47330,0.16,294.0,-124.746002,40.527000,20.25,California,POINT (-124.74600 40.52700),109,40 - 60


In [27]:
gdf_sismos_con_zonas.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 142593 entries, 0 to 142592
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   id                         142593 non-null  object  
 1   Magnitud                   142593 non-null  float64 
 2   Place                      142490 non-null  object  
 3   Primer Registro            142593 non-null  object  
 4   Ultimo registro            142593 non-null  object  
 5   Felt                       23869 non-null   float64 
 6   cdi                        23869 non-null   float64 
 7   mmi                        6038 non-null    float64 
 8   Posibilidad tsunami        142593 non-null  int64   
 9   Importancia del evento     142593 non-null  int64   
 10  ids                        142593 non-null  object  
 11  nst                        120414 non-null  float64 
 12  Dist Horizontal epicentro  93371 non-null   float64 
 13  RMS   

In [28]:
gdf_sismos_con_zonas.drop(['Primer Registro', 'Ultimo registro', 'Felt', 'cdi', 'mmi',
       'Posibilidad tsunami', 'ids', 'nst',
       'RMS', 'Brecha azimutal', 'Dist Horizontal epicentro',
       'estado', 'index_right'], axis = 1, inplace=True)

In [29]:
gdf_sismos_con_zonas.head()

,id,Magnitud,Place,Importancia del evento,Longitud,Latitud,Profundidad,geometry,ValueRange
0,nc73888821,2.83,58km WNW of Petrolia,123,-124.955833,40.425335,-0.37,POINT (-124.95583 40.42533),40 - 60
1,nc73888746,3.24,2km NNE of Almanor,169,-121.167833,40.237167,6.02,POINT (-121.16783 40.23717),20 - 30
2,ok2023jmlz,2.98,8 km NW of Prague,139,-96.749667,35.536000,7.81,POINT (-96.74967 35.53600),60 - 80
3,nc73888511,2.60,3km NNE of Prattville,104,-121.142833,40.232167,6.76,POINT (-121.14283 40.23217),20 - 30
4,nc73888396,2.63,41km W of Ferndale,106,-124.746002,40.527000,20.25,POINT (-124.74600 40.52700),40 - 60


In [30]:
gdf_zonas.head()

,ValueRange,geometry
0,40 - 60,"POLYGON ((-103.28787 31.11885, -103.30000 31.1..."
1,30 - 40,"POLYGON ((-103.15886 31.30000, -103.15537 31.2..."
2,20 - 30,"POLYGON ((-103.10000 31.37753, -103.08522 31.3..."
3,16 - 20,"POLYGON ((-103.13594 31.46406, -103.12207 31.4..."
4,12 - 16,"POLYGON ((-103.06452 31.51452, -103.05000 31.5..."


In [31]:
gdf_sismos_con_zonas.isnull().sum()

id                          0
Magnitud                    0
Place                     103
Importancia del evento      0
Longitud                    0
Latitud                     0
Profundidad                28
geometry                    0
ValueRange                  0
dtype: int64

In [32]:
gdf_sismos_con_zonas.dropna(inplace = True)

## Aplicar método supervisado SVC  
El método de aprendizaje supervisado Support Vector Classification (SVC) es un algoritmo utilizado para clasificar datos en dos o más clases. El SVC se basa en la idea de encontrar un hiperplano óptimo que separe las clases en el espacio de características.

### Preparar las características (atributos)

In [33]:
X = gdf_sismos_con_zonas[['Magnitud', 'Longitud', 'Latitud', 'Profundidad']].values

### Preparar las etiquetas (clases)

In [34]:
y = gdf_sismos_con_zonas['ValueRange'].values

In [35]:
#separo el dataframe en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
#normalizo los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
#creo el modelo de aprendizaje supervisado
model = SVC()
#entreno el modelo SVC:
model.fit(X_train, y_train)

SVC()

In [38]:
#evaluo el modelo
y_pred = model.predict(X_test)

Evaluar la precisión de tu modelo utilizo métricas como la precisión, el recall, el F1-score y la matriz de confusión.

Precisión: La precisión es la proporción de instancias clasificadas correctamente como positivas (verdaderos positivos) sobre el total de instancias clasificadas como positivas (verdaderos positivos más falsos positivos).

In [39]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

# Calcular la precisión, el recall y el F1-score para cada clase (si es un problema multiclase)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [40]:
accuracy

0.5327975292177026

In [41]:
precision

0.5269818577080402

In [42]:
recall

0.5327975292177026

In [43]:
f1

0.5024343066467809

In [44]:
# Generar la matriz de confusión
confusion_mtx = confusion_matrix(y_test, y_pred)

print(confusion_mtx)

[[ 989   77    3  494  260   33   51   23    3   81    0   26]
 [ 154  299    1  796  227   26  188    4    1    9    0   24]
 [   7    0 1197    9    1  185    0   11    0   15   23    0]
 [  27  129    3 2462  821   33 1154    4    2    5    0  287]
 [   6   11   10  601 1656   36 2091    0    3    6    0   79]
 [  96    4   40   39    6  774    3   47    0   58    5    0]
 [   1    0    2  253  905   45 4234    0   13   16    0  415]
 [ 108    8    2   42    7  100   17  254    0  124    1    1]
 [   0    2    0    9   18    0  597    1   18    0    0 1204]
 [ 163   19    4  236   55   62   34   56    0  291    0   10]
 [   0    2   21   15    2    4    0    0    0    2  276    0]
 [   0    0    0   11    2    0  375    0   10    0    0 2731]]


Idea: separar sismo peligroso o no peligroso